In [1]:
import os
import joblib
from pathlib import Path
from time import time
import pandas as pd
import numpy as np
from tqdm import tqdm
from model_settings import ms
from quantlib_pricers import barriers

ms.find_root(Path())
ms.__dict__

modelsdir = os.path.join(ms.root,ms.trained_models)
folders = [f for f in os.listdir(modelsdir) if f.find('Legacy')==-1]
folders

['2024-11-15 164145753233 cboe spx relative asian',
 '2024-11-15 164315824655 cboe spx relative barrier']

In [2]:
model_folder = folders[1]
pricename = str(model_folder[model_folder.rfind(' ')+1:])+'_price'
model_dir = os.path.join(modelsdir,model_folder)
pickle = [f for f in os.listdir(model_dir) if f.endswith('.pkl') and f.find(model_folder)!=-1][0]
model = joblib.load(os.path.join(model_dir,pickle))
est = model['model']
df = model['train_data'].iloc[:,:-4].copy().reset_index(drop=True)
df = df.drop(columns=[c for c in df.columns if c.find(pricename)!=-1])
pricename

'barrier_price'

In [ ]:
bar = tqdm(total=df.shape[0])
for i,row in df.iterrows():
    tic = time()
    df.at[i, 'MLP'] = est.predict(df.iloc[[i]].loc[:, model['feature_set']])
    MLP = time() - tic
    df.at[i,'MLP_cpu'] = MLP 
    bar.update(1)
bar.close()

  4%|██▉                                                                       | 10400/266781 [00:27<11:22, 375.39it/s]

In [ ]:
df = barriers.df_barrier_price(df)
df.to_csv(f"{ms.timetag()} {pricename[pricename.find('_'):]} performance.csv")